In [ ]:
# This script is written by ixScience and can be found here 
# https://github.com/ixScience/fourier_neural_operator/blob/master/data_generation/navier_stokes/ns_2d.py
!pip install torch scipy tqdm

import torch
import torch.nn.functional as F
import math
from timeit import default_timer
import scipy.io
from scipy.ndimage import zoom
import numpy as np
from tqdm.autonotebook import tqdm

pathToProject = 'SciREX/examples/sciml/datasets/super_resolution/ns2d'
gpu = False # Set to true if CUDA is available

from random_fields import GaussianRF

In [3]:
#w0: initial vorticity
#f: forcing term
#visc: viscosity (1/Re)
#T: final time
#delta_t: internal time-step for solve (descrease if blow-up)
#record_steps: number of in-time snapshots to record
def navier_stokes_2d(w0, f, visc, T, delta_t=1e-4, record_steps=1):

    #Grid size - must be power of 2
    N = w0.size()[-1]

    #Maximum frequency
    k_max = math.floor(N/2.0)

    #Number of steps to final time
    steps = math.ceil(T/delta_t)

    #Initial vorticity to Fourier space
    w_h = torch.fft.rfft2(w0)

    #Forcing to Fourier space
    f_h = torch.fft.rfft2(f)

    #If same forcing for the whole batch
    if len(f_h.size()) < len(w_h.size()):
        f_h = torch.unsqueeze(f_h, 0)

    #Record solution every this number of steps
    record_time = math.floor(steps/record_steps)

    #Wavenumbers in y-direction
    k_y = torch.cat((torch.arange(start=0, end=k_max, step=1, device=w0.device), torch.arange(start=-k_max, end=0, step=1, device=w0.device)), 0).repeat(N,1)
    #Wavenumbers in x-direction
    k_x = k_y.transpose(0,1)

    #Truncate redundant modes
    k_x = k_x[..., :k_max + 1]
    k_y = k_y[..., :k_max + 1]

    #Negative Laplacian in Fourier space
    lap = 4*(math.pi**2)*(k_x**2 + k_y**2)
    lap[0,0] = 1.0
    #Dealiasing mask
    dealias = torch.unsqueeze(torch.logical_and(torch.abs(k_y) <= (2.0/3.0)*k_max, torch.abs(k_x) <= (2.0/3.0)*k_max).float(), 0)

    #Saving solution and time
    sol = torch.zeros(*w0.size(), record_steps, device=w0.device)
    sol_t = torch.zeros(record_steps, device=w0.device)

    #Record counter
    c = 0
    #Physical time
    t = 0.0
    for j in tqdm(range(steps)):
        #Stream function in Fourier space: solve Poisson equation
        psi_h = w_h / lap

        #Velocity field in x-direction = psi_y
        q = 2. * math.pi * k_y * 1j * psi_h
        q = torch.fft.irfft2(q, s=(N, N))

        #Velocity field in y-direction = -psi_x
        v = -2. * math.pi * k_x * 1j * psi_h
        v = torch.fft.irfft2(v, s=(N, N))

        #Partial x of vorticity
        w_x = 2. * math.pi * k_x * 1j * w_h
        w_x = torch.fft.irfft2(w_x, s=(N, N))

        #Partial y of vorticity
        w_y = 2. * math.pi * k_y * 1j * w_h
        w_y = torch.fft.irfft2(w_y, s=(N, N))

        #Non-linear term (u.grad(w)): compute in physical space then back to Fourier space
        F_h = torch.fft.rfft2(q*w_x + v*w_y)

        #Dealias
        F_h = dealias* F_h

        #Crank-Nicolson update
        w_h = (-delta_t*F_h + delta_t*f_h + (1.0 - 0.5*delta_t*visc*lap)*w_h)/(1.0 + 0.5*delta_t*visc*lap)

        #Update real time (used only for recording)
        t += delta_t

        if (j+1) % record_time == 0:
            #Solution in physical space
            w = torch.fft.irfft2(w_h, s=(N, N))

            #Record solution and time
            sol[...,c] = w
            sol_t[c] = t

            c += 1


    return sol, sol_t

if gpu:
    device = torch.device('cuda') 
else:
    device = torch.device('cpu')

#Resolution
s = 1024

#Number of solutions to generate
N = 10

#Set up 2d GRF with covariance parameters
GRF = GaussianRF(2, s, alpha=2.5, tau=7, device=device)

#Forcing function: 0.1*(sin(2pi(x+y)) + cos(2pi(x+y)))
t = torch.linspace(0, 1, s+1, device=device)
t = t[0:-1]

X,Y = torch.meshgrid(t, t, indexing='ij')
f = 0.1*(torch.sin(2*math.pi*(X + Y)) + torch.cos(2*math.pi*(X + Y)))

#Number of snapshots from solution

#Inputs
record_steps = 50
a = torch.zeros(N, s, s)
#Solutions
u = torch.zeros(N, s, s, record_steps)

#Solve equations in batches (order of magnitude speed-up)

#Batch size
bsize = 10

T = 50.0
viscosity = 1e-3
delta_t = 1e-4


# steps = math.ceil(T/delta_t)
# recordTime = math.floor(steps/record_steps)


c = 0
t0 =default_timer()
for j in range(N//bsize):

    #Sample random feilds
    w0 = GRF.sample(bsize)

    #Solve NS
    sol, sol_t = navier_stokes_2d(w0, f, viscosity, T, delta_t, record_steps)

    a[c:(c+bsize),...] = w0
    u[c:(c+bsize),...] = sol

    c += bsize
    t1 = default_timer()
    print(j, c, t1-t0)


scipy.io.savemat('ns_data_test.mat', mdict={'a': a.cpu().numpy(), 'u': u.cpu().numpy(), 't': sol_t.cpu().numpy()})
np.save('u_test', u)
u.shape

100%|██████████| 500000/500000 [18:43<00:00, 445.20it/s]


0 10 1123.3529052190715


torch.Size([10, 1024, 1024, 50])

In [ ]:
def create_lr_hr_pairs(u, lr_size=16, hr_size=128, method='bicubic'):
    N, H, W, T = u.shape
    
    # Initialize output arrays
    lr_data = np.zeros((N, lr_size, lr_size, T))
    hr_data = np.zeros((N, hr_size, hr_size, T))
    
    print(f"Creating LR ({lr_size}×{lr_size}) and HR ({hr_size}×{hr_size}) pairs...")
    
    u_torch = torch.from_numpy(u).float()
    
    for i in range(N):
        for t in range(T):
            # Extract single frame: (1024, 1024)
            frame = u_torch[i, :, :, t].unsqueeze(0).unsqueeze(0)  # (1, 1, 1024, 1024)
            
            # Downsample to HR size (128×128)
            if method == 'bicubic':
                hr_frame = F.interpolate(frame, size=(hr_size, hr_size), 
                                        mode='bicubic', align_corners=False)
            elif method == 'bilinear':
                hr_frame = F.interpolate(frame, size=(hr_size, hr_size), 
                                        mode='bilinear', align_corners=False)
            elif method == 'area':
                hr_frame = F.interpolate(frame, size=(hr_size, hr_size), 
                                        mode='area')
            else:
                hr_frame = F.interpolate(frame, size=(hr_size, hr_size), 
                                        mode='nearest')
            
            hr_data[i, :, :, t] = hr_frame.squeeze().numpy()
            
            # Downsample to LR size (16×16)
            if method == 'bicubic':
                lr_frame = F.interpolate(frame, size=(lr_size, lr_size), 
                                        mode='bicubic', align_corners=False)
            elif method == 'bilinear':
                lr_frame = F.interpolate(frame, size=(lr_size, lr_size), 
                                        mode='bilinear', align_corners=False)
            elif method == 'area':
                lr_frame = F.interpolate(frame, size=(lr_size, lr_size), 
                                        mode='area')
            else:
                lr_frame = F.interpolate(frame, size=(lr_size, lr_size), 
                                        mode='nearest')
            
            lr_data[i, :, :, t] = lr_frame.squeeze().numpy()
        
        if (i + 1) % 2 == 0:
            print(f"Processed {i + 1}/{N} samples")
    
    return lr_data, hr_data


# Load the data
dataDict = scipy.io.loadmat('ns_data_test.mat')
a, t, u = dataDict['a'], dataDict['t'], dataDict['u']

print(f"Original data shape: {u.shape}")


lr_data, hr_data = create_lr_hr_pairs(u, lr_size=16, hr_size=128, method='bicubic')

print(f"LR data shape: {lr_data.shape}")
print(f"HR data shape: {hr_data.shape}")


np.save(pathToProject + 'ns_lr_16x16.npy', lr_data)
np.save(pathToProject + 'ns_hr_128x128.npy', hr_data)

# if needed
scipy.io.savemat(pathToProject + '_ns_sr_pairs.mat', {
    'lr': lr_data,
    'hr': hr_data,
    't': t,
    'a': a
})

print("\nData saved successfully")
print(f"LR: {pathToProject}_ns_lr_16x16.npy with shape {lr_data.shape}")
print(f"HR: {pathToProject}_ns_hr_128x128.npy with shape {hr_data.shape}")

Original data shape: (10, 1024, 1024, 50)
Creating LR (16×16) and HR (128×128) pairs...
Processed 2/10 samples
Processed 4/10 samples
Processed 6/10 samples
Processed 8/10 samples
Processed 10/10 samples
LR data shape: (10, 16, 16, 50)
HR data shape: (10, 128, 128, 50)

Data saved successfully
LR: /home/abhilash/Projects/SciREX/examples/sciml/datasets/super_resolution/ns2dns_lr_16x16.npy with shape (10, 16, 16, 50)
HR: /home/abhilash/Projects/SciREX/examples/sciml/datasets/super_resolution/ns2dns_hr_128x128.npy with shape (10, 128, 128, 50)
